In [1]:
try:
    import inotify
except ImportError as e:
    !pip install inotify

In [2]:
from dask.distributed import Client
import subprocess as subp
import os
import dask

In [3]:
import inotify.adapters

def create_ompi_server(uri_file):

    cmd = ["ompi-server", "--no-daemonize","-r", uri_file]
    cmdStr = "exec " + " ".join(cmd)
    
    i = inotify.adapters.Inotify()
    
    with open(uri_file, 'w') as f:
        pass
    
    
    i.add_watch(uri_file)

    proc = subp.Popen(cmdStr, shell=True)

    for event in i.event_gen(yield_nones=False):
        (_, type_names, path, filename) = event
        if "IN_CLOSE_WRITE" in type_names:
            break
    
    
    i.remove_watch(uri_file)

    
    mpiServer = proc
    import time
    with open(uri_file, "r") as fp:
        mpiServerUri = fp.read().rstrip()
        
    return (mpiServer, mpiServerUri)

In [4]:
uriFile = "ompi.server.uri"

mpiServer, mpiServerUri = create_ompi_server(uriFile)
os.environ["OMPI_MCA_pmix_server_uri"] = mpiServerUri

In [5]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()

In [6]:
c = Client(cluster)
c

Client Scheduler: tcp://127.0.0.1:39731 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 50.39 GB


In [7]:
import hello_mpi

In [8]:
def parse_host_port(address):
    if '://' in address:
        address = address.rsplit('://', 1)[1]
    host, port = address.split(':')
    port = int(port)
    return host, port

In [9]:
def run(workerId, nWorkers, ompiServerUri):
    if ompiServerUri is None:
        raise Exception("ompiServerUri is mandatory!")
    os.environ["OMPI_MCA_pmix_server_uri"] = ompiServerUri
    w = dask.distributed.get_worker()
    print("Hello World! from ip=%s worker=%s/%d uri=%s" % \
          (w.address, w.name, nWorkers, ompiServerUri))
    print("Worker=%s finished" % w.name)
    
    a = hello_mpi.World(workerId, nWorkers)
    a.init()

    return a

In [10]:
workers = list(zip(map(lambda x: parse_host_port(x), c.has_what().keys()), range(len(c.has_what().keys()))))
f = [c.submit(run, idx, 2, mpiServerUri, workers=[worker]) for worker, idx in workers]


In [11]:
import random

def get_rank(world, r):
    return world.rank()

def finalize(world):
    world.finalize()

In [12]:
ranks = [c.submit(get_rank, a, random.random()) for a in f]
for i in ranks:
    print(str(i.result()))

0
1


In [13]:
[c.submit(finalize, a) for a in f]

[<Future: status: pending, key: finalize-29b8800f27d460d460363a125011ab3d>,
 <Future: status: pending, key: finalize-368dc52407099ed523252fd85db31617>]

In [14]:
mpiServer.kill()
os.remove(uriFile)